# Data Load & Preprocessing 

구텐베르크 프로잭트 기반 cleaned data load :
https://huggingface.co/datasets/incredible45/Gutenberg-BookCorpus-Cleaned-Data-English

간단한 규칙 기반 장면 분할:
    - 연속된 빈 줄(3개 이상) 또는 ---/*** 같은 구분선을 강한 분리 신호로 사용
    - 문단 단위로 묶으며, 따옴표가 매우 많은 블록은 별도 장면으로 취급

.pkl 로 저장

In [ ]:
import re
import os
from tqdm import tqdm
import pickle
from datasets import load_dataset
from itertools import islice

# Data Preprocessing

In [20]:
def split_into_scenes(text):
    """
    간단한 규칙 기반 장면 분할:
    - 연속된 빈 줄(3개 이상) 또는 ---/*** 같은 구분선을 강한 분리 신호로 사용
    - 문단 단위로 묶으며, 따옴표가 매우 많은 블록은 별도 장면으로 취급
    """
    # 강한 분리: 빈 줄 3개 이상 또는 연속 구분선
    blocks = re.split(r'\n{3,}|[-*#]{3,}', text)
    scenes = []
    buffer = []

    for blk in blocks:
        blk = blk.strip()
        if not blk:
            continue

        # 따옴표(큰/작은 따옴표) 개수로 대화 중심 블록 감지
        quote_count = blk.count('"') + blk.count("'")

        # 따옴표가 많으면 현재 버퍼를 장면으로 묶고 새 장면 시작
        if quote_count > 6:
            if buffer:
                scenes.append("\n\n".join(buffer).strip())
                buffer = []
            scenes.append(blk)
        else:
            buffer.append(blk)

    if buffer:
        scenes.append("\n\n".join(buffer).strip())

    return scenes

## Data Load

In [23]:
dataset = load_dataset(
    "incredible45/Gutenberg-BookCorpus-Cleaned-Data-English",
    split="train",
    streaming=True
)
count = 0
MAX_BOOKS = 5  # 저장할 최대 책 권수

# 저장 폴더 생성
save_dir = "books_pickle"
os.makedirs(save_dir, exist_ok=True)


for example in tqdm(dataset, desc="Processing books"):
    content = example.get("content", "")
    if len(content.strip()) < 2000:  # 최소 길이 필터
        continue

    # 장면 분할
    scenes = split_into_scenes(content)

    # 저장할 데이터 구조
    record = {
        "book_id": example.get("book_id", ""),
        "title": example.get("title", ""),
        "author": example.get("author", ""),
        "scenes": scenes
    }

    # 파일명 안전하게 만들기
    title_safe = example.get("title", "untitled").replace(" ", "_").replace("/", "_")
    filename = f"{save_dir}/{title_safe}_{example.get('book_id','')}.pkl"

    # pickle 저장
    with open(filename, "wb") as f:
        pickle.dump(record, f)

    count += 1
    if count >= MAX_BOOKS:  # 최대 저장 권수 도달 시 종료
        break

print(f"✅ 저장 완료! 총 {count}권 저장됨. 폴더: {save_dir}")

Processing books: 2729it [00:44, 60.87it/s] 


KeyboardInterrupt: 